In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

t1=0
t2=60
carpeta='MUNDO'
zona='MUNDO'

#1. INICIAR RHO
def init_rho():
    rho=np.zeros((len(lat2),len(lon2)))
    valor_rho=0.1
    for y in range(len(lat2)): 
        for x in range(len(lon2)):
            if mask2[y,x]==1.0:
                rho[y,x]=valor_rho
            else: 
                rho[y,x]=0.0
    return rho
#----------------------------------

#el tercer parametro: kleft=0, kbottom=1, kright=2, kup=3, karea=4
#acabada   

def partition_Oflow(O, Ow, Oo, y, x, k, localrho):
    if (O>0.0):
        Ow[y,x,k]=localrho[y,x]*O
        Oo[y,x,k]=O-Ow[y,x,k]
    else:
        Ow[y,x,k]=0.0
        Oo[y,x,k]=0.0
    return Ow, Oo

def guess_output_flows(rho):
    thisrho=rho
    qx=np.zeros((len(lat2),len(lon2)))
    qy=np.zeros((len(lat2),len(lon2)))
    Ow=np.zeros((len(lat2),len(lon2),4))
    Oo=np.zeros((len(lat2),len(lon2),4))
    
    for y in range(len(lat2)):
            for x in range(len(lon2)):
                R=6371000
                Atetha=math.radians(1.0)
                Alambda=math.radians(1.0)
                tethaup=math.radians(lat2[y]+0.5)
                tethadown=math.radians(lat2[y]-0.5)
                #lambdar=math.radians(lon2[x])
                Ay=R*Atetha
                Axup=R*math.cos(tethaup)*Alambda
                Axdown=R*math.cos(tethadown)*Alambda
            #LEFT
                ux=-1.0
                uy=0.0
                qx[y,x]=(Qx[y,x]+Qx[y+1,x])/2.0
                qy[y,x]=(Qy[y,x]+Qy[y+1,x])/2.0
        
                Ol=(ux*qx[y,x] + uy*qy[y,x])*Ay
                Ow, Oo=partition_Oflow(Ol, Ow, Oo, y, x, kleft, thisrho)
          
             #UP
                ux=0.0
                uy=1.0
                qx[y,x]=(Qx[y,x]+Qx[y,x+1])/2.0
                qy[y,x]=(Qy[y,x]+Qy[y,x+1])/2.0
                Ou=(ux*qx[y,x] + uy*qy[y,x])*Axup
                Ow, Oo=partition_Oflow(Ou, Ow, Oo, y, x, kup, thisrho)
             #RIGHT
                ux=1.0
                uy=0.0
                qx[y,x]=(Qx[y,x+1]+Qx[y+1,x+1])/2.0
                qy[y,x]=(Qy[y,x+1]+Qy[y+1,x+1])/2.0
                Or=(ux*qx[y,x] + uy*qy[y,x])*Ay
                Ow, Oo=partition_Oflow(Or,Ow, Oo, y, x,  kright, thisrho)
             #BOTTOM
                ux=0.0
                uy=-1.0
                qx[y,x]=(Qx[y+1,x]+Qx[y+1,x+1])/2.0
                qy[y,x]=(Qy[y+1,x]+Qy[y+1,x+1])/2.0
                Ob=(ux*qx[y,x] + uy*qy[y,x])*Axdown
                Ow, Oo=partition_Oflow(Ob,Ow, Oo, y, x, kbottom, thisrho)
                
    return Ow,Oo  


def output_to_inflow(Ow, Oo):
# las mascaras que no son 0
        Iw=np.zeros((len(lat2),len(lon2)))
        Io=np.zeros((len(lat2),len(lon2)))
        for y in range(len(lat2)):
            for x in range(len(lon2)):
                
                if mask2[y,x]==0.0:
                    continue
                    
                if x==0:
                # Left border 
                    Iw[y,x]=Iw[y,x]+Ow[y,x+1,kleft]
                    Io[y,x]=Io[y,x]+Oo[y,x+1,kleft]
                    
                    Iw[y,x]=Iw[y,x]+Ow[y,len(lon2)-1,kright]
                    Io[y,x]=Io[y,x]+Oo[y,len(lon2)-1,kright]
                    
                    if (y>0):
                        Iw[y,x]=Iw[y,x]+Ow[y-1,x,kbottom]
                        Io[y,x]=Io[y,x]+Oo[y-1,x,kbottom]
         
                    if (y<(len(lat2)-1)):
                        Iw[y,x]=Iw[y,x]+Ow[y+1,x,kup]
                        Io[y,x]=Io[y,x]+Oo[y+1,x,kup]
        

                if x==(len(lon2)-1):
                # Right border 
                    Iw[y,x]=Iw[y,x]+Ow[y,0,kleft]
                    Io[y,x]=Io[y,x]+Oo[y,0,kleft]
                    
                    Iw[y,x]=Iw[y,x]+Ow[y,x-1,kright]
                    Io[y,x]=Io[y,x]+Oo[y,x-1,kright]
                    
                    if (y>0):
                        Iw[y,x]=Iw[y,x]+Ow[y-1,x,kbottom]
                        Io[y,x]=Io[y,x]+Oo[y-1,x,kbottom]
         
                    if (y<(len(lat2)-1)):
                        Iw[y,x]=Iw[y,x]+Ow[y+1,x,kup]
                        Io[y,x]=Io[y,x]+Oo[y+1,x,kup]
                        
                elif y==0:
                # Up border 
                    Iw[y,x]=Iw[y,x]+Ow[y+1,x,kup]
                    Io[y,x]=Io[y,x]+Oo[y+1,x,kup]
                    if (x>0):
                        Iw[y,x]=Iw[y,x]+Ow[y,x-1,kright]
                        Io[y,x]=Io[y,x]+Oo[y,x-1,kright]
         
                    if (x<(len(lon2)-1)):
                        Iw[y,x]=Iw[y,x]+Ow[y,x+1,kleft]
                        Io[y,x]=Io[y,x]+Oo[y,x+1,kleft] 
                        
                elif y==(len(lat2)-1):
                # Bottom border 
                    Iw[y,x]=Iw[y,x]+Ow[y-1,x,kbottom]
                    Io[y,x]=Io[y,x]+Oo[y-1,x,kbottom]
                    if (x>0):
                        Iw[y,x]=Iw[y,x]+Ow[y,x-1,kright]
                        Io[y,x]=Io[y,x]+Oo[y,x-1,kright]
         
                    if (x<(len(lon2)-1)):
                        Iw[y,x]=Iw[y,x]+Ow[y,x+1,kleft]
                        Io[y,x]=Io[y,x]+Oo[y,x+1,kleft]
           
                
                
                else:
                # inside 
                    Iw[y,x]=Iw[y,x]+Ow[y,x-1,kright]
                    Io[y,x]=Io[y,x]+Oo[y,x-1,kright]
                    
                    Iw[y,x]=Iw[y,x]+Ow[y-1,x,kbottom]
                    Io[y,x]=Io[y,x]+Oo[y-1,x,kbottom]
                    
                    Iw[y,x]=Iw[y,x]+Ow[y+1,x,kup]
                    Io[y,x]=Io[y,x]+Oo[y+1,x,kup]
                    
                    Iw[y,x]=Iw[y,x]+Ow[y,x+1,kleft]
                    Io[y,x]=Io[y,x]+Oo[y,x+1,kleft]
                    
        return Iw, Io



def add_border_inflows(Io):
    qx=np.zeros((len(lat2),len(lon2)))
    qy=np.zeros((len(lat2),len(lon2)))
    
    # LEFT 
    x=0
    ux= 1.0 # INFLOWS!!! 
    uy= 0.0
    for y in range(len(lat2)):
            qx[y,x]=(Qx[y,x]+Qx[y+1,x])/2.0
            qy[y,x]=(Qy[y,x]+Qy[y+1,x])/2.0
            F=(ux*qx[y,x] +uy*qy[y,x])*Ay
            if (F>0.):
                Io[y,x]=Io[y,x]+F
                
        # UP
    y=0
    ux= 0.0 # INFLOWS!!! 
    uy= -1.0
    for x in range(len(lon2)):
            tethaup=math.radians(lat2[y]+0.5)
            Axup=R*(math.cos(tethaup))*Alambda
            qx[y,x]=(Qx[y,x]+Qx[y,x+1])/2.0
            qy[y,x]=(Qy[y,x]+Qy[y,x+1])/2.0
            F=(ux*qx[y,x] +uy*qy[y,x])*Axup
            if (F>0.):
                Io[y,x]=Io[y,x]+F
        #RIGHT            
    x=len(lon2)-1
    ux= -1.0 # INFLOWS!!! 
    uy= 0.0
    for y in range(len(lat2)):
            qx[y,x]=(Qx[y,x+1]+Qx[y+1,x+1])/2.0
            qy[y,x]=(Qy[y,x+1]+Qy[y+1,x+1])/2.0
            F=(ux*qx[y,x] +uy*qy[y,x])*Ay
            if (F>0.):
                Io[y,x]=Io[y,x]+F
                
        #BOTTOM            
    y=len(lat2)-1
    ux= 0.0 # INFLOWS!!! 
    uy= 1.0
    for x in range(len(lon2)):
            tethadown=math.radians(lat2[y]-0.5)
            Axdown=R*(math.cos(tethadown))*Alambda
            qx[y,x]=(Qx[y+1,x]+Qx[y+1,x+1])/2.0
            qy[y,x]=(Qy[y+1,x]+Qy[y+1,x+1])/2.0
            F=(ux*qx[y,x] +uy*qy[y,x])*Axdown
            if (F>0.):
                Io[y,x]=Io[y,x]+F
    return Io
                             
                    
def update_rho(rho):  
    #make a guess of Ow and Oo from Qx,Qy and existing rho estimation
    Ow, Oo=   guess_output_flows(rho)
    #Convert output flows to input flows (partitioned)
    Iw,Io=   output_to_inflow(Ow, Oo) 
    #Consider border inflows as external flows
    Io=(add_border_inflows(Io))
    
    #From the estimated Io,Iw, get a new estimation for rho
    rho2=np.zeros((len(lat2),len(lon2)))
    for y in range(len(lat2)): #len(rho[y,0])
        for x in range(len(lon2)):#len(rho[0,x])
            #We assume that points not masked are "exterior" and we don't mind their recycling ratio.
            if mask2[y,x]==0.0:
                continue
            rho2[y,x]=(Iw[y,x]+Eflow[y,x])/(Iw[y,x]+Io[y,x]+Eflow[y,x])
    return rho2 
#delta rho
def distance(rho, rho2):
    dist=0.0
    for y in range(len(lat2)):
        for x in range(len(lon2)):
            td=abs(rho[y,x]-rho2[y,x])
            if td>dist:
                dist=td
            
    return dist
#tiempo 2015-2019

for t in range (t1,t2):

    #mes='julio_2010_2021'
#mask = np.loadtxt('mask_%s.txt'%(zona))
#mask2 = np.loadtxt('mask2_%s.txt'%(zona))Eflow = np.loadtxt('media_Eflow_%s_%s.txt'%(zona,mes))
#Eflow_mm = np.loadtxt('media_Eflow_mm_%s_%s.txt'%(zona,mes))
#P=np.loadtxt('media_P_%s_%s.txt'%(zona,mes))
#lon2=np.loadtxt('lon2.txt')
#lat2=np.loadtxt('lat2.txt')
#Qx=np.loadtxt('media_Qx_%s_%s.txt'%(zona,mes))
#Qy=np.loadtxt('media_Qy_%s_%s.txt'%(zona,mes))

    mask = np.loadtxt('C:/Users/jonre/TFG/%s/mask_%s.txt'%(carpeta,zona))
    mask2 = np.loadtxt('C:/Users/jonre/TFG/%s/mask2_%s.txt'%(carpeta,zona))
    Eflow = np.loadtxt('C:/Users/jonre/TFG/%s/Eflow_%s_%d.txt'%(carpeta,zona,t))
    P=np.loadtxt('C:/Users/jonre/TFG/%s/P_%s_%d.txt'%(carpeta,zona,t))
    lon2=np.loadtxt('C:/Users/jonre/TFG/%s/lon2_%s.txt'%(carpeta,zona))
    lat2=np.loadtxt('C:/Users/jonre/TFG/%s/lat2_%s.txt'%(carpeta,zona))
    Qx=np.loadtxt('C:/Users/jonre/TFG/%s/Qx_%s_%d.txt'%(carpeta,zona,t))
    Qy=np.loadtxt('C:/Users/jonre/TFG/%s/Qy_%s_%d.txt'%(carpeta,zona,t))

    kleft=0
    kbottom=1
    kright=2
    kup=3

    R=6371000
    Atetha=math.radians(1.0)
    Alambda=math.radians(1.0)
    Ay=R*Atetha
    
    #------------------------------------
    #-------PROGRAMA PRINCIPAL-----------
    #------------------------------------
    itermax=5000
    tolerance=1e-6
    rho=init_rho() #bien

#dependen de la iteracion
    for i in range(1000):
        rho2=update_rho(rho)
        dist=distance(rho, rho2) 
        if dist<tolerance:
            print('se ha roto en',i)
            break
        rho=rho2
        #print(i)
        #print(dist)
    
    #print(rho)
    print('max', np.amax(rho), 'tiemp', t)

    #np.savetxt('rho_%s_s%.txt'%(zona,mes), rho)
    np.savetxt('C:/Users/jonre/TFG/%s/rho_%s_%d.txt'%(carpeta,zona,t), rho)
    
    
    
    
    
    
    
    #ACABA EL PROGRAMA
    #-----------------------------------------------------------------
    #-----------------------------------------------------------------
    #-----------------------------------------------------------------
    #-----------------------------------------------------------------
    #SACAR LOS VALORES DE INFLOW Y OUTFLOW
   
    Ow, Oo=   guess_output_flows(rho)
    #Convert output flows to input flows (partitioned)
    Iw,Io=   output_to_inflow(Ow, Oo) 
    #Consider border inflows as external flows
    Io=(add_border_inflows(Io))
    
    Oo_mm= np.zeros((len(lat2),len(lon2)))
    Ow_mm= np.zeros((len(lat2),len(lon2)))
    Iw_mm= np.zeros((len(lat2),len(lon2)))
    Io_mm= np.zeros((len(lat2),len(lon2))) 
    
    for y in range(len(lat2)): 
        for x in range(len(lon2)):
            Atetha=math.radians(1.0)
            Alambda=math.radians(1.0)
            tethamedio=math.radians(lat2[y])
            #A=(R**2)*(math.sin(tethamedio+(Atetha/2.0))-math.sin(tethamedio-(Atetha/2.0)))*Alambda
            A=(R**2)*2*math.cos(tethamedio)*math.sin(Atetha/2)*Alambda
            Io_mm[y,x] = (Io[y,x]*86400)/(A)
            Iw_mm[y,x] = (Iw[y,x]*86400)/(A)
            for i in range (4):
                Oo_mm[y,x]=Oo_mm[y,x]+Oo[y,x,i]
                Ow_mm[y,x]=Ow_mm[y,x]+Ow[y,x,i]
            Oo_mm[y,x]=(Oo_mm[y,x]*86400)/(A)
            Ow_mm[y,x]=(Ow_mm[y,x]*86400)/(A)
            

    np.savetxt('C:/Users/jonre/TFG/%s/Oo_mm_%s_%d.txt'%(carpeta,zona,t), Oo_mm)
    np.savetxt('C:/Users/jonre/TFG/%s/Ow_mm_%s_%d.txt'%(carpeta,zona,t), Ow_mm)
    np.savetxt('C:/Users/jonre/TFG/%s/Io_mm_%s_%d.txt'%(carpeta,zona,t), Io_mm)
    np.savetxt('C:/Users/jonre/TFG/%s/Iw_mm_%s_%d.txt'%(carpeta,zona,t), Iw_mm)
        
    #------------------------------------------------------------
                

se ha roto en 92
max 0.8683381885834034 tiemp 0
se ha roto en 93
max 0.9940390270536269 tiemp 1
se ha roto en 79
max 0.9785827130866839 tiemp 2
se ha roto en 115
max 0.9592297392983147 tiemp 3
se ha roto en 101
max 0.6762029009234238 tiemp 4
se ha roto en 91
max 0.8621882259070741 tiemp 5
se ha roto en 101
max 0.6983313934146044 tiemp 6
se ha roto en 101
max 0.7571485603109424 tiemp 7
se ha roto en 96
max 0.7728969142850997 tiemp 8
se ha roto en 94
max 0.8378739582736213 tiemp 9
se ha roto en 85
max 0.8416321231284524 tiemp 10
se ha roto en 91
max 0.737169768335721 tiemp 11
se ha roto en 121
max 0.9933574508325618 tiemp 12
se ha roto en 95
max 0.8861413389711361 tiemp 13
se ha roto en 80
max 0.918312939562809 tiemp 14
se ha roto en 94
max 0.8033583665898562 tiemp 15
se ha roto en 130
max 0.8551599330532941 tiemp 16
se ha roto en 94
max 0.7722693789692704 tiemp 17
se ha roto en 97
max 0.7871224515530346 tiemp 18
se ha roto en 101
max 0.7467768234711797 tiemp 19
se ha roto en 92
max 0.89